In [2]:
import xarray as xr
import os
import numpy as np
import pandas as pd

In [3]:
res = 2.8125
local_dir = f"E:\\weatherbench\\{res}deg"

In [ ]:
def get_daily_predictor_reading_for_year(predictor, lat, lon, year):
    d = xr.open_dataset(os.path.join(local_dir, predictor, f"{predictor}_{res}deg", f"{predictor}_{year}_{res}deg.nc"))
    one_slice = dict(lat=slice(lat, lat+1), lon=slice(lon, lon+1))
    data = d.loc[one_slice].to_dataframe()
    temps = data.query(f"lon=={lon} and lat=={lat}")['t2m']
    # hourly temperature: each day covers 24 entries. Take the daily average over 24hrs
    one_year_daily=np.array([np.mean(temps[i:i+24]) for i in range(0, len(temps), 24)])
    return one_year_daily[:365].reshape((1, 365))

In [5]:
d=xr.open_dataset("E:\\weatherbench\\2.8125deg\\toa_incident_solar_radiation\\toa_incident_solar_radiation_2.8125deg\\toa_incident_solar_radiation_1979_2.8125deg.nc")

In [49]:
x=d.coords

In [53]:
lats = [lt.item() for lt in d['lat']]

In [ ]:
def detrend(lat, lon, years, nombor):
    """
    Extract the mean feature value for each day in a year over the period between y1 and y2 inclusive.
    """
    y1, y2 = years
    data = None
    for y in range(y1, y2+1):
        more_data = get_daily_temp_for_year(lat, lon, y)
        if data is None:
            data = more_data
        else:
            data = np.concatenate((data, more_data), axis=0)
    np.savetxt(f"daily_averages{nombor}.txt", data, delimiter=',')
    # means is an array where mean[0] is the feature value average over the years between y1 and y2 on the 
    # on the first day of a year
    means = np.array([np.array([data[i, j] for i in range(40)]).mean() for j in range(365)])
    detrended = data - means
    np.savetxt(f"detrended{nombor}.txt", linearize(detrended), delimiter=',')